# Getting Started with SingleStoreDB and Python

This notebook is a simple demonstration of the SingleStore Python package capabilities. It is by no means exhaustive of all features, but is 
meant as an introduction to the types of things you can do. There are three Python packages utilized which represent layers of capabilities
that build on each other. The three packages are:

* SingleStoreDB Python - the base layer which implements the [Python DB-API](https://peps.python.org/pep-0249/) as well as the 
  [SingleStoreDB Cloud Management API](https://docs.singlestore.com/managed-service/en/user-and-workspace-administration/management-api.html).
* SingleStoreDB SQLAlchemy Dialect - this is a [SQLAlchemy](https://www.sqlalchemy.org) plugin which allows you to interact with SingleStoreDB using SQLAlchemy.
* SingleStoreDB Ibis Backend - this package implements a backend for the [Ibis](https://ibis-project.org/) framework which contains a generalized DataFrame
  API similar to the [pandas](https://pandas.pydata.org) DataFrame API that works with databases, data warehouses, and other backends (such as pandas itself).
  
Since the SingleStoreDB Ibis backend is built on the SingleStore SQLAlchemy dialect which is in turn built on the SingleStoreDB Python package,
we'll start right out using Ibis to exercise all of them at once.

In [1]:
import ibis
import pandas as pd

We'll set up a few options to make the notebook nicer to use, including the display width of DataFrames and enabling the interactive features
of the Ibis package. The primary purpose of the `interactive` option is so that you don't have to manually run the `execute` method on expressions
to see the results. In our notebook, the `execute` method will be called implicitly each time an Ibis expression is rendered by the notebook.

In [2]:
pd.options.display.max_colwidth = 120
ibis.options.verbose = False
ibis.options.interactive = True

## Make a connection using Ibis

Connections to Ibis are made through the `ibis.singlestoredb` method. The connection parameters can be given using either the suggested Python DB-API
parameters (i.e., host=, user=, port=, etc.) or as a SQLAlchemy URL (e.g., user:password@host:port/database). In this case, we are getting the username and password from the environment variable `SINGLESTOREDB_USER` and `SINGLESTOREDB_PASSWORD`.

In [4]:
conn = ibis.singlestoredb.connect('root:ibis@127.0.0.1:8306/x_db')

ImportError: cannot import name 'dtype_from_sqlalchemy' from 'ibis.backends.base.sql.alchemy.datatypes' (/home/ksmith/src/ibis/ibis/backends/base/sql/alchemy/datatypes.py)

SingleStoreDB also has an HTTP API which is wrapped in a Python DB-API compliant form, so it can be used as well by specifying an HTTP(s) URL and
the corresponding port number.

In [4]:
# conn = ibis.singlestoredb.connect('http://127.0.0.1:8000/x_db')

## Phone review data example

Our first example will use customer review data for phones. It includes information such as a reviewer name, the text of the review, a short summary
of the review, a numeric rating from 1 to 5, and the date of the review. We'll first read the data into a local pandas DataFrame to do some
cleanup, then upload it to the database to demonstrate the SingleStoreDB package capabilities

### Read sample data into pandas DataFrame

Here we're simply displaying the top of the CSV file containing the data to see the column names and the first line of data.

In [5]:
%%sh 

head -2 phones.csv

reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
"A30TL5EWN6DFXT","120401325X","christina","[0, 0]","They look good and stick good! I just don't like the rounded shape because I was always bumping it and Siri kept popping up and it was irritating. I just won't buy a product like this again","4.0","Looks Good","1400630400","05 21, 2014"


The data is read using the `read_csv` function.

In [6]:
phones_df = pd.read_csv('phones.csv')
phones_df.head(3)

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,A30TL5EWN6DFXT,120401325X,christina,"[0, 0]",They look good and stick good! I just don't like the rounded shape because I was always bumping it and Siri kept pop...,4.0,Looks Good,1400630400,"05 21, 2014"
1,ASY55RVNIL0UD,120401325X,emily l.,"[0, 0]",These stickers work like the review says they do. They stick on great and they stay on the phone. They are super sty...,5.0,Really great product.,1389657600,"01 14, 2014"
2,A2TMXE2AFO7ONB,120401325X,Erica,"[0, 0]",These are awesome and make my phone look so stylish! I have only used one so far and have had it on for almost a yea...,5.0,LOVE LOVE LOVE,1403740800,"06 26, 2014"


We can see the column types using the `info` method. As you can see, there are two date columns, neither of which is in a convenient
format for use as a date. We'll first create an actual datetime column from those and drop the original columns.

In [7]:
phones_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3921 entries, 0 to 3920
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   reviewerID      3921 non-null   object 
 1   asin            3921 non-null   object 
 2   reviewerName    3921 non-null   object 
 3   helpful         3921 non-null   object 
 4   reviewText      3921 non-null   object 
 5   overall         3921 non-null   float64
 6   summary         3920 non-null   object 
 7   unixReviewTime  3921 non-null   int64  
 8   reviewTime      3921 non-null   object 
dtypes: float64(1), int64(1), object(7)
memory usage: 275.8+ KB


### Upload the data to the server

Now that we have some data to work with, we can upload it to the server using the `create_table` method. This method allows you to upload
data from a local DataFrame or materialize a table from an Ibis table expression.

In [8]:
conn.create_table?

Signature:
conn.create_table(
    name: 'str',
    expr: 'Optional[Union[pd.DataFrame, ir.TableExpr]]' = None,
    schema: 'Optional[sch.Schema]' = None,
    database: 'Optional[str]' = None,
    force: 'bool' = False,
    storage_type: 'Optional[str]' = None,
    schema_overrides: 'Optional[Union[Dict[str, str], sch.Schema]]' = None,
) -> 'ir.Table'
Docstring:
Create a new table.

Parameters
----------
name : str
    Name of the new table.
expr : Schema or DataFrame, optional
    An Ibis table expression or pandas DataFrame that will be used to
    extract the schema and the data of the new table. If not provided,
    ``schema`` must be given.
schema : Schema, optional
    The schema for the new table. Only one of ``schema`` or ``expr`` can be
    provided.
database : str, optional
    Name of the database where the table will be created, if not the
    default.
force : bool, optional
    Check whether a table exists before creating it
storage_type : str, optional
    The storage type

The code below will create the table in the server from the given DataFrame. The data types for the database table will be inferred from
the DataFrame data types. The `force=True` is merely used here in case you run the notebook code multiple times against the same
database. It will cause an existing `phones` table to be overwritten if it already exists.

In [9]:
phones_tbl = conn.create_table('phones', phones_df, force=True)
phones_tbl.head(3)

┏━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━┓
┃ reviewerID     ┃ asin       ┃ reviewerName                  ┃ helpful ┃ reviewText                                                                       ┃ overall ┃ summary              ┃ unixReviewTime ┃ reviewTime  ┃
┡━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━┩
│ string         │ string     │ string                        │ string  │ string                                                                           │ float64 │ string               │ int64          │ string      │
├────────────────┼────────────┼───────────────────────────────┼─────────┼──────────────────────────────────────────────────────────────────────────────────┼─────────┼──────────────────────┼────────────────┼─────────────┤
│ A3AFELPYTZH90T │ 3998899561 │ Baja Alan                     │ [2, 3]  │ This is the first battery case I have had for my Galaxy S4. The S4 fits very we… │     5.0 │ A Winner             │     1384905600 │ 11 20, 2013 │
│ ACU3LCRX4A8RV  │ 3998899561 │ Zonaldo Reefey Zonaldo Reefey │ [2, 3]  │ It works great. Doesn't heat up like crazy like the other ones I got, and cheap… │     5.0 │ SUPER DUPER QUALITY! │     1377388800 │ 08 25, 2013 │
│ A2U5NF3IH4YVKH │ 6073894996 │ ebony winslow                 │ [0, 0]  │ After a week only one side works                                                 │     2.0 │ Two Stars            │     1404345600 │ 07 3, 2014  │
└────────────────┴────────────┴───────────────────────────────┴─────────┴──────────────────────────────────────────────────────────────────────────────────┴─────────┴──────────────────────┴────────────────┴─────────────┘

We can get information about the table in the server by using the `info` method, just as you would with a local pandas DataFrame.

In [10]:
phones_tbl.info()

┏━━━━━━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━┓
┃ name           ┃ type    ┃ nullable ┃ nulls ┃ non_nulls ┃ null_frac ┃ pos  ┃
┡━━━━━━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━┩
│ string         │ string  │ boolean  │ int64 │ int64     │ float64   │ int8 │
├────────────────┼─────────┼──────────┼───────┼───────────┼───────────┼──────┤
│ reviewerID     │ string  │ True     │     0 │      3921 │    0.0000 │    0 │
│ asin           │ string  │ True     │     0 │      3921 │    0.0000 │    1 │
│ reviewerName   │ string  │ True     │     0 │      3921 │    0.0000 │    2 │
│ helpful        │ string  │ True     │     0 │      3921 │    0.0000 │    3 │
│ reviewText     │ string  │ True     │     0 │      3921 │    0.0000 │    4 │
│ overall        │ float64 │ True     │     0 │      3921 │    0.0000 │    5 │
│ summary        │ string  │ True     │     1 │      3920 │    0.0003 │    6 │
│ unixReviewTime │ int64   │ True     │     0 │      3921 │    0.0000 │    7 │
│ reviewTime     │ string  │ True     │     0 │      3921 │    0.0000 │    8 │
└────────────────┴─────────┴──────────┴───────┴───────────┴───────────┴──────┘

Here is the `CREATE TABLE` statement resulting from the data upload step. The `show` attribute of the connection gives access to all of
the `SHOW` commands in the database such as `functions`, `create_function`, `status`, `databases`, and so on.

In [11]:
conn.show.create_table('phones')

Name,CreateTable
phones,"CREATE TABLE `phones` ( `reviewerID` text CHARACTER SET utf8 COLLATE utf8_general_ci, `asin` text CHARACTER SET utf8 COLLATE utf8_general_ci, `reviewerName` text CHARACTER SET utf8 COLLATE utf8_general_ci, `helpful` text CHARACTER SET utf8 COLLATE utf8_general_ci, `reviewText` text CHARACTER SET utf8 COLLATE utf8_general_ci, `overall` double DEFAULT NULL, `summary` text CHARACTER SET utf8 COLLATE utf8_general_ci, `unixReviewTime` bigint(20) DEFAULT NULL, `reviewTime` text CHARACTER SET utf8 COLLATE utf8_general_ci, SORT KEY `__UNORDERED` () , SHARD KEY () ) AUTOSTATS_CARDINALITY_MODE=OFF AUTOSTATS_HISTOGRAM_MODE=OFF AUTOSTATS_SAMPLING=OFF SQL_MODE='STRICT_ALL_TABLES'"


### Convert review date / time columns

This data set has a less than optimal way of storing the dates of the reviews. It has two columns that contain the same information, and neither
of them is convenient to work with. Let's create a new column that contains the date in a timestamp column. To do that, we simply cast the
existing `unixReviewTime` column to a timestamp type. We'll also use the `name` method to assign a name to the resulting column expression.

In [12]:
date = phones_tbl.unixReviewTime.cast('timestamp').name('date')
date

┏━━━━━━━━━━━━━━━━━━━━━┓
┃ date                ┃
┡━━━━━━━━━━━━━━━━━━━━━┩
│ timestamp           │
├─────────────────────┤
│ 2013-11-20 00:00:00 │
│ 2013-08-25 00:00:00 │
│ 2014-07-03 00:00:00 │
│ 2013-04-07 00:00:00 │
│ 2014-05-07 00:00:00 │
│ 2013-11-20 00:00:00 │
│ 2011-08-19 00:00:00 │
│ 2013-07-31 00:00:00 │
│ 2013-10-12 00:00:00 │
│ 2014-04-09 00:00:00 │
│ …                   │
└─────────────────────┘

The `overall` rating column was uploaded as floating point data, but it only contains integers. Let's cast that to an integer.

In [13]:
overall = phones_tbl.overall.cast('int').name('overall')
overall

┏━━━━━━━━━┓
┃ overall ┃
┡━━━━━━━━━┩
│ int64   │
├─────────┤
│       5 │
│       5 │
│       2 │
│       5 │
│       5 │
│       5 │
│       5 │
│       5 │
│       5 │
│       5 │
│       … │
└─────────┘

Now that we have our two processed columns, we can remove the old columns we don't need anymore and insert our
new columns expressions with the cleaned data.

In [14]:
columns = [x for x in phones_tbl.columns if 'Time' not in x and x != 'overall'] + [overall, date]

In [15]:
phones_tbl = phones_tbl[columns] 
phones_tbl

┏━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┓
┃ reviewerID     ┃ asin       ┃ reviewerName                  ┃ helpful  ┃ reviewText                                                                       ┃ summary                                                     ┃ overall ┃ date                ┃
┡━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━┩
│ string         │ string     │ string                        │ string   │ string                                                                           │ string                                                      │ int64   │ timestamp           │
├────────────────┼────────────┼───────────────────────────────┼──────────┼──────────────────────────────────────────────────────────────────────────────────┼─────────────────────────────────────────────────────────────┼─────────┼─────────────────────┤
│ A2B8HBZPIFYRV1 │ 6073894996 │ pissed off                    │ [0, 0]   │ works great and charges ipads, tablets, smartphones as well as bluetooth speake… │ Good little charger                                         │       4 │ 2014-01-26 00:00:00 │
│ A205L2POTHUAR7 │ 8199406933 │ Ana                           │ [0, 0]   │ very good charger, it woks fine, no complaints!!! I would recommend it!! It was… │ Good charger!                                               │       4 │ 2013-11-11 00:00:00 │
│ A1AGQVCNRY2G8G │ 8288853439 │ MDB                           │ [4, 4]   │ This Charger is awesome! First of all I wasn't expecting it until between 12/12… │ Galaxy S4 Charger                                           │       5 │ 2013-12-11 00:00:00 │
│ A1MMMVGC7A7P1Y │ 8288862993 │ alex                          │ [0, 0]   │ It works great  thus  far no issues.  The only thing i might have limited was a… │ this is great buy and very useful                           │       3 │ 2014-04-12 00:00:00 │
│ A1EA86UX957SE2 │ 8288878881 │ June Rey June Rey             │ [0, 0]   │ First off, the charger works just as it should and all the ports stay connected… │ Works perfect; size and shape were different than expected. │       3 │ 2012-12-04 00:00:00 │
│ A1Y3YLHX4JWK6R │ 8288878881 │ Paul Monroe Jr.               │ [0, 0]   │ I would like to start off by saying that this product is what I ordered and I h… │ Correct Product BUT...                                      │       3 │ 2013-07-06 00:00:00 │
│ ACU3LCRX4A8RV  │ 878968897X │ Zonaldo Reefey Zonaldo Reefey │ [0, 0]   │ This is a very well built and designed case,with a very cool and attractive col… │ What a pleasant color                                       │       5 │ 2014-05-08 00:00:00 │
│ A8AJS1DW7L3JJ  │ 9819958431 │ Agata Majchrzak               │ [0, 3]   │ The raised edges protect the screens from falls. The battery case looks stylish… │ BOMBA                                                       │       5 │ 2014-06-08 00:00:00 │
│ A1847XXVEA8QUK │ 9861203192 │ Daniel Poole                  │ [25, 25] │ Bought this for my new AT&T; Galaxy SII, the case arrived super fast, its fits … │ Awesome case and cheap too                                  │       5 │ 2011-10-07 00:00:00 │
│ A4799JODK16F7  │ 9866137252 │ bucketofroses                 │ [0, 0]   │ I made a good choice whenI thought of buying this, its very safe, & not to bulk… │ Very good case.                                             │       5 │ 2014-01-21 00:00:00 │
│ …              │ …          │ …                             │ …        │ …                                                                                │ …                                                           │ 

The `phones_tbl` object now contains both references to actual columns in the `phones` table as well as computed expressions.
It can be thought of as a client-side view object, but still works the same way as our original table. We can use the `info`
method to show the schema information.

In [16]:
phones_tbl.info()

┏━━━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━┓
┃ name         ┃ type      ┃ nullable ┃ nulls ┃ non_nulls ┃ null_frac ┃ pos  ┃
┡━━━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━┩
│ string       │ string    │ boolean  │ int64 │ int64     │ float64   │ int8 │
├──────────────┼───────────┼──────────┼───────┼───────────┼───────────┼──────┤
│ reviewerID   │ string    │ True     │     0 │      3921 │    0.0000 │    0 │
│ asin         │ string    │ True     │     0 │      3921 │    0.0000 │    1 │
│ reviewerName │ string    │ True     │     0 │      3921 │    0.0000 │    2 │
│ helpful      │ string    │ True     │     0 │      3921 │    0.0000 │    3 │
│ reviewText   │ string    │ True     │     0 │      3921 │    0.0000 │    4 │
│ summary      │ string    │ True     │     1 │      3920 │    0.0003 │    5 │
│ overall      │ int64     │ True     │     0 │      3921 │    0.0000 │    6 │
│ date         │ timestamp │ True     │     0 │      3921 │    0.0000 │    7 │
└──────────────┴───────────┴──────────┴───────┴───────────┴───────────┴──────┘

It's also possible to access the schema information for traversing progamatically using the `schema` method of the table.

In [17]:
phones_tbl.schema()

ibis.Schema {
  reviewerID    string
  asin          string
  reviewerName  string
  helpful       string
  reviewText    string
  summary       string
  overall       int64
  date          timestamp
}

### Create a table from a table expression

As mentioned earlier, the `create_table` method can be used to create tables in the database from Ibis table
expressions. That means that we can materialize our new table into the database if we so desire. This simply
selects data from an expression into a new table, no work is being done on the client side.

In [18]:
tbl_2 = conn.create_table('phones2', phones_tbl, force=True)
tbl_2

┏━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┓
┃ reviewerID     ┃ asin       ┃ reviewerName            ┃ helpful ┃ reviewText                                                                       ┃ summary                       ┃ overall ┃ date                ┃
┡━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━┩
│ string         │ string     │ string                  │ string  │ string                                                                           │ string                        │ int64   │ timestamp           │
├────────────────┼────────────┼─────────────────────────┼─────────┼──────────────────────────────────────────────────────────────────────────────────┼───────────────────────────────┼─────────┼─────────────────────┤
│ A3AFELPYTZH90T │ 3998899561 │ Baja Alan               │ [2, 3]  │ This is the first battery case I have had for my Galaxy S4. The S4 fits very we… │ A Winner                      │       5 │ 2013-11-20 00:00:00 │
│ A3HWOEJ6BK7B7M │ 6073894996 │ Wei                     │ [0, 0]  │ good product at low price.purchased this looking for a smaller charger and I lo… │ good product at low price     │       5 │ 2014-05-07 00:00:00 │
│ A1STLXG6FB22P2 │ 8199406933 │ Natica C.               │ [0, 0]  │ Great product. Use this with my Galaxy S4. At this price I bought 2 more. One f… │ Great charger and great price │       5 │ 2013-11-20 00:00:00 │
│ A2DHFLDTV8BMWK │ 8288878881 │ Jean Polycart           │ [0, 0]  │ This detachable sansung galaxy s2 travel charger is simply good. It works perfe… │ Jean P. review.               │       5 │ 2014-04-09 00:00:00 │
│ A1BHU6FHSC0BJF │ B00001W0EQ │ Skyline                 │ [1, 1]  │ Excited about this based on good reviews, but did not have the same experience.… │ static, poor sound quality    │       2 │ 2012-04-16 00:00:00 │
│ A2NP9CGUSFP22E │ B00002X29G │ BG Bob G                │ [1, 2]  │ Wish it was magnetized but it's sturdy & has the common pieces I need when I'm … │ Nice set                      │       4 │ 2013-02-14 00:00:00 │
│ A285PI5661DZST │ B00002X29G │ Linda J. Shores         │ [1, 1]  │ I needed a new screwdriver set to work on laptops with, as most of my old ones … │ They're screwdrivers          │       5 │ 2014-03-12 00:00:00 │
│ A1N4P89OQJ0LN0 │ B000246XQE │ William Reading         │ [4, 5]  │ I picked up this headset the other day and I'm relatively pleased with it. Desp… │ Not bad at all                │       4 │ 2005-03-05 00:00:00 │
│ A2M1GWZ1A7FAUT │ B0002WRGHG │ bigbearsfan bigbearsfan │ [8, 9]  │ This headset is actually not a bad deal but it is not a great deal either.  The… │ Functional but not great..... │       2 │ 2005-10-24 00:00:00 │
│ A28NAD0W2DU7ZR │ B0002WRGHG │ Eric Goodrich odinsdad  │ [1, 1]  │ Since there is no rubber to gasket your ear, noisy environments can make it har… │ Works but can be hard to hear │       3 │ 2007-09-28 00:00:00 │
│ …              │ …          │ …                       │ …       │ …                                                                                │ …                             │       … │ …                   │
└────────────────┴────────────┴─────────────────────────┴─────────┴──────────────────────────────────────────────────────────────────────────────────┴───────────────────────────────┴─────────┴─────────────────────┘

In [19]:
conn.show.tables()

Name
phones
phones2
titanic


In [20]:
tbl_2.info()

┏━━━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━┓
┃ name         ┃ type      ┃ nullable ┃ nulls ┃ non_nulls ┃ null_frac ┃ pos  ┃
┡━━━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━┩
│ string       │ string    │ boolean  │ int64 │ int64     │ float64   │ int8 │
├──────────────┼───────────┼──────────┼───────┼───────────┼───────────┼──────┤
│ reviewerID   │ string    │ True     │     0 │      3921 │    0.0000 │    0 │
│ asin         │ string    │ True     │     0 │      3921 │    0.0000 │    1 │
│ reviewerName │ string    │ True     │     0 │      3921 │    0.0000 │    2 │
│ helpful      │ string    │ True     │     0 │      3921 │    0.0000 │    3 │
│ reviewText   │ string    │ True     │     0 │      3921 │    0.0000 │    4 │
│ summary      │ string    │ True     │     1 │      3920 │    0.0003 │    5 │
│ overall      │ int64     │ True     │     0 │      3921 │    0.0000 │    6 │
│ date         │ timestamp │ True     │     0 │      3921 │    0.0000 │    7 │
└──────────────┴───────────┴──────────┴───────┴───────────┴───────────┴──────┘

In [21]:
conn.show.create_table('phones2')

Name,CreateTable
phones2,"CREATE TABLE `phones2` ( `reviewerID` text CHARACTER SET utf8 COLLATE utf8_general_ci, `asin` text CHARACTER SET utf8 COLLATE utf8_general_ci, `reviewerName` text CHARACTER SET utf8 COLLATE utf8_general_ci, `helpful` text CHARACTER SET utf8 COLLATE utf8_general_ci, `reviewText` text CHARACTER SET utf8 COLLATE utf8_general_ci, `summary` text CHARACTER SET utf8 COLLATE utf8_general_ci, `overall` bigint(20) DEFAULT NULL, `date` datetime DEFAULT NULL, SORT KEY `__UNORDERED` () , SHARD KEY () ) AUTOSTATS_CARDINALITY_MODE=OFF AUTOSTATS_HISTOGRAM_MODE=OFF AUTOSTATS_SAMPLING=OFF SQL_MODE='STRICT_ALL_TABLES'"


For this example, we'll drop this table and use our original object.

In [22]:
conn.drop_table('phones2')

### DataFrame operations

Now that we have a table in the database and a DataFrame-like obect connected to it, let's do some operations with the data. For this example, 
we'll enable the `verbose` option in Ibis. This will display the SQL code that is sent to the server when evaluating the expression. In this 
example, we are building a sentence from various columns in the data table. We are using the reviwer name (which is capitalized), the summary,
and the overall rating (which is cast to a string). All of these are concatenated together using the `+` as you normally would when
concatenating strings in Python.

In [23]:
# Enable display of SQL that is executed in the background
ibis.options.verbose = True

texpr = phones_tbl.reviewerName.capitalize() + \
        ' says "' + \
        phones_tbl.summary + \
        '" and gives it ' + \
        phones_tbl.overall.cast('string') + \
        ' stars.'
texpr

SELECT concat(concat(concat(concat(concat(concat(ucase(left(t0."reviewerName", :left_1)), substring(t0."reviewerName", :substring_1)), :param_1), t0.summary), :param_2), CAST(t0.overall AS TEXT)), :param_3) AS "StringConcat()" 
FROM (SELECT t1."reviewerID" AS "reviewerID", t1.asin AS asin, t1."reviewerName" AS "reviewerName", t1.helpful AS helpful, t1."reviewText" AS "reviewText", t1.summary AS summary, CAST(t1.overall AS BIGINT) AS overall, convert_tz(from_unixtime(t1."unixReviewTime"), :convert_tz_1, :convert_tz_2) AS date 
FROM phones AS t1) AS t0
 LIMIT :param_4


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ StringConcat()                                                                  ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ string                                                                          │
├─────────────────────────────────────────────────────────────────────────────────┤
│ Baja Alan says "A Winner" and gives it 5 stars.                                 │
│ Zonaldo Reefey Zonaldo Reefey says "SUPER DUPER QUALITY!" and gives it 5 stars. │
│ Ebony winslow says "Two Stars" and gives it 2 stars.                            │
│ Jason M. Faulkner JMF says "good" and gives it 5 stars.                         │
│ Wei says "good product at low price" and gives it 5 stars.                      │
│ Natica C. says "Great charger and great price" and gives it 5 stars.            │
│ Steve says "Awesome" and gives it 5 stars.                                      │
│ Joleen Hunt says "Works great!" and gives it 5 stars.                           │
│ Amazon Customer says ":) :) :)" and gives it 5 stars.                           │
│ Jean Polycart says "Jean P. review." and gives it 5 stars.                      │
│ …                                                                               │
└─────────────────────────────────────────────────────────────────────────────────┘

As you can see from the SQL output above, the sentence is constructed using `concat` in the query. Capitalization is done using `ucase` and `substring`. 
The `:param_#` portions are used by Ibis to inject literal Python values from the expression. And, you'll see that the result table contains a column
of string values that are the result of the expression.

#### String operations

There are many string operations available in Ibis. Many of them correspond to operations on pandas DataFrames and / or Python string methods.
The code below prints a list of all methods on string columns.

In [24]:
print(' '.join([x for x in dir(texpr) if not x.startswith('_')]))

approx_median approx_nunique arbitrary argmax argmin as_table asc ascii_str authority between capitalize case cases cast coalesce collect compile concat contains convert_base count cume_dist cummax cummin dense_rank desc describe endswith equals execute file fillna find find_in_set first fragment get_name greatest group_concat has_name hash hashbytes head host identical_to ilike initcap isin isnull join lag last lead least left length like lower lpad lstrip max min mode name notin notnull nth ntile nullif nunique op over path percent_rank pipe protocol query rank re_extract re_replace re_search repeat replace reverse right rlike rpad rstrip split startswith strip substitute substr summary to_csv to_pandas to_parquet to_pyarrow to_pyarrow_batches to_timestamp topk translate type typeof unbind upper userinfo value_counts visualize


#### Operations on other column types

Other data types have different methods that can be called. For example, this is the list of methods on date/time columns.

In [25]:
print(' '.join([x for x in dir(phones_tbl.date) if not x.startswith('_')]))

add approx_median approx_nunique arbitrary argmax argmin as_table asc between case cases cast coalesce collect compile count cume_dist cummax cummin date day day_of_week day_of_year dense_rank desc describe epoch_seconds equals execute fillna first get_name greatest group_concat has_name hash head hour identical_to isin isnull lag last lead least max millisecond min minute mode month name notin notnull nth ntile nullif nunique op over percent_rank pipe quarter radd rank rsub second strftime sub substitute summary time to_csv to_pandas to_parquet to_pyarrow to_pyarrow_batches topk truncate type typeof unbind value_counts visualize week_of_year year


Here are the methods for numeric columns.

In [26]:
print(' '.join([x for x in dir(phones_tbl.overall) if not x.startswith('_')]))

abs acos add approx_median approx_nunique arbitrary argmax argmin as_table asc asin atan atan2 between bit_and bit_or bit_xor bucket case cases cast ceil clip coalesce collect compile convert_base corr cos cot count cov cume_dist cummax cummean cummin cumsum deg2rad degrees dense_rank desc describe div equals execute exp fillna first floor floordiv get_name greatest group_concat has_name hash head histogram identical_to isin isnull label lag last lead least ln log log10 log2 max mean min mod mode mul name negate notin notnull nth ntile nullif nullifzero nunique op over percent_rank pipe point pow quantile rad2deg radd radians rank rdiv rfloordiv rmod rmul round rpow rsub sign sin sqrt std sub substitute sum summary tan to_csv to_interval to_pandas to_parquet to_pyarrow to_pyarrow_batches to_timestamp topk type typeof unbind value_counts var visualize zeroifnull


In [27]:
ibis.options.verbose = False

### Filter reviews containing "iphone 4"

Now that we've seen how to construct expressions that can be used in the database, let's use that expression for use in further queries.
The expression below takes the result of our previous expression, lower-cases it, then checks for the occurrence of the substring
"iphone 4". This will return a column of booleans which can be used to filter our original table, just like with a pandas DataFrame.

In [28]:
has_iphone_4 = texpr.lower().contains('iphone 4')
has_iphone_4

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ StringContains(Lowercase(StringConcat()), 'iphone 4') ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ boolean                                               │
├───────────────────────────────────────────────────────┤
│ False                                                 │
│ False                                                 │
│ False                                                 │
│ False                                                 │
│ False                                                 │
│ False                                                 │
│ False                                                 │
│ False                                                 │
│ False                                                 │
│ False                                                 │
│ …                                                     │
└───────────────────────────────────────────────────────┘

First, let's use the filter expression to see how many of our generated sentences contain "iphone 4". We do this by using our previous
expression as an indexing expression to our original table object (e.g., `tbl[filter-expr]`). This will return only the rows where
the filter expression was `True`. We can then count the number of rows using the `count` method.

In [29]:
phones_tbl[has_iphone_4].count()

7

Now we can print selected columns from the filtered table. We can also sort them according to the `overall` column as shown below.

In [30]:
phones_tbl[has_iphone_4]['reviewerName', 'helpful', 'overall'].order_by(ibis.desc('overall'))

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━┓
┃ reviewerName        ┃ helpful ┃ overall ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━┩
│ string              │ string  │ int64   │
├─────────────────────┼─────────┼─────────┤
│ Dennis Dennis in NH │ [1, 1]  │       5 │
│ patrice m rogoza    │ [2, 3]  │       5 │
│ Stephan S.          │ [2, 2]  │       5 │
│ Mufasa              │ [6, 6]  │       5 │
│ Monica              │ [0, 0]  │       5 │
│ Mike                │ [0, 0]  │       4 │
│ Mark T.             │ [1, 1]  │       4 │
└─────────────────────┴─────────┴─────────┘

So far we've uploaded data to the database using `create_table` and a local DataFrame, we've constructed Python expressions that get translated
to SQL and executed on the server, used those expressions in filtering operations, and selected columns from and sorted the results of
the filtering operation. This covers a small number of the abilities of the table and column objects created by Ibis. For additional methods
on the various types, see the [Ibis API reference](https://ibis-project.org/docs/3.2.0/api/expressions/).

## XML book data example

We are going to look at another example utilitizing XML data now. This will demonstrate integration with user-defined functions (UDF) in the server.

### Define book information in XML

As before, we'll use a local pandas DataFrame to construct a table of data and upload it to the server.

In [31]:
xml_df = pd.DataFrame([
    (1, '<book><author>James McGovern</author><title>XQuery Kick Start</title></book>'),
    (2, '<book><author>Aurelien Geron</author><title>Hands-On Machine Learning</title></book>'),
    (3, '<book><author>Luciano Ramalho</author><title>Fluent Python</title></book>'),
    (4, '<book><author>Scott Chacon</author><title>Pro Git</title></book>'),
], columns=['id', 'xml'])

In [32]:
xml_tbl = conn.create_table('books', xml_df, force=True)
xml_tbl.head()

┏━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ id    ┃ xml                                                                              ┃
┡━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ int64 │ string                                                                           │
├───────┼──────────────────────────────────────────────────────────────────────────────────┤
│     4 │ <book><author>Scott Chacon</author><title>Pro Git</title></book>                 │
│     2 │ <book><author>Aurelien Geron</author><title>Hands-On Machine Learning</title></… │
│     3 │ <book><author>Luciano Ramalho</author><title>Fluent Python</title></book>        │
│     1 │ <book><author>James McGovern</author><title>XQuery Kick Start</title></book>     │
└───────┴──────────────────────────────────────────────────────────────────────────────────┘

### Build and push `eval_xpath` WASM function

We are going to define a UDF using WASM and the [SingleStore WASM toolkit](https://github.com/singlestore-labs/singlestore-wasm-toolkit).
If you want to develop your own UDFs using WASM, you'll want to 
[check out the tutorial](https://singlestore-labs.github.io/singlestore-wasm-toolkit/html/Tutorial-Overview.html). For our purposes here,
we'll use one of the examples from the toolkit.

The `examples/rust/xpath` directory includes a sample UDF that evaluates XPath expressions against strings containing XML documents.
[XPath](https://www.w3.org/TR/xpath/) is a file path-like syntax that can be used to traverse and extract values from XML content.
To build and install the `eval_xpath` function from the examples, you need to run the following commands.

```
> git clone git@github.com:singlestore-labs/singlestore-wasm-toolkit.git
> cd singlestore-wasm-toolkit/examples/rust/xpath
> cargo wasi build --lib --release
> pushwasm udf --prompt --conn mysql://root@127.0.0.1:3306/x_db --wit xpath.wit --wasm target/wasm32-wasi/release/xpath.wasm --name eval_xpath
```

Because the function was created before the UDF existed in the server, you'll need to synchronize the functions
to the client manually. However, any UDFs that exist when the connection is created will already be available.

In [33]:
conn.sync_functions()

We can use the `functions` method on the `show` object to display the information for all of the available functions.

In [34]:
conn.show.functions()

Name,FunctionType,Definer,DataFormat,RuntimeType,Link
eval_xpath,User Defined Function,root@%,,Wasm,


We can use the `create_function` method of the `show` object to display the `CREATE FUNCTION` syntax of the function.

In [35]:
conn.show.create_function('eval_xpath')

Name,SQLMode,CreateFunction,CharacterSetClient,CollationConnection
eval_xpath,STRICT_ALL_TABLES,"CREATE OR REPLACE FUNCTION `eval_xpath`(xml text CHARACTER SET utf8mb4 COLLATE utf8mb4_general_ci NOT NULL, expr text CHARACTER SET utf8mb4 COLLATE utf8mb4_general_ci NOT NULL) RETURNS text CHARACTER SET utf8mb4 COLLATE utf8mb4_general_ci NOT NULL DEFINER = 'root'@'%' AS WASM ABI CANONICAL GROW TO 134217728 FROM [...] USING EXPORT 'eval-xpath';",utf8,utf8_general_ci


### Using the UDF

Since the first parameter of the `eval_xpath` is a string, the function can be called on string columns in the table object.
In the code below, we are using the XPath expression `/book/author` to pull the author entry out of each of the XML
documents in the `xml` column. This will return a string expression which we then use to create a filtering expression
by looking for a capital letter 'G' in the author name returned by the XPath expression.

In [36]:
g_authors = xml_tbl.xml.eval_xpath('/book/author').contains('G')
g_authors

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ StringContains(EvalXpath(xml, '/book/author'), 'G') ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ boolean                                             │
├─────────────────────────────────────────────────────┤
│ False                                               │
│ True                                                │
│ True                                                │
│ False                                               │
└─────────────────────────────────────────────────────┘

As you can see in the result above, two of the author's names contained a capital 'G'. We will now use that filtering expression
to subset the rows in the `xml_tbl` object and return the title of each of the books associated with those authors.

In [37]:
title = xml_tbl[g_authors].xml.eval_xpath('/book/title').name('Title')
title

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Title                     ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ !string                   │
├───────────────────────────┤
│ XQuery Kick Start         │
│ Hands-On Machine Learning │
└───────────────────────────┘